In [10]:
from rainymotion import models, metrics, utils
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime,timedelta
import calendar
from netCDF4 import Dataset
import joblib
import os
print(os.getcwd())


/Users/jiang/github_all/rainymotion/docs/notebooks


In [11]:
kakuho_folder = "/Users/jiang/data/rain_kakuho"
radar_folder = "/Users/jiang/data/jma_radar"

In [77]:
time_step = 5 * 60 # seconds
threshold = 0.1  # mm/h
lead_time = list(range(0,65,5))

for j in range(0,24):
    dt_now = datetime(2020,1,22,j,0)
    dt_pre = dt_now - timedelta(seconds = time_step)
    dt_12  = dt_now + timedelta(seconds = time_step * 12)
    yday_pre = dt_pre.strftime('%Y_%m_%d')
    yday_now = dt_now.strftime('%Y_%m_%d')
    yday_12  = dt_12.strftime('%Y_%m_%d')

    # load ground truth data
    daily_database = {}
    inputs = np.zeros(shape = (2,1000,1000), dtype = np.float16)
    if yday_pre not in daily_database:
        daily_database[yday_pre] = joblib.load(os.path.join(radar_folder, 
                                                            f"jma_radar_uint8_{yday_pre}.joblib")) 
    if yday_12 not in daily_database:
        daily_database[yday_12]  = joblib.load(os.path.join(radar_folder,
                                                            f"jma_radar_uint8_{yday_12}.joblib")) 
    # load data into optical flow model
    sequence_in_day = int(dt_pre.hour*12 + (dt_pre.minute)/5)    
    inputs[0,:,:] = daily_database[yday_pre][sequence_in_day]/10.0
    inputs[1,:,:] = daily_database[yday_now][(sequence_in_day + 1)%288]/10.0  
    model = models.Dense()    
    model.input_data = inputs
    model.lead_steps = 13
    nowcast = model.run()  # shape (13, 1000, 1000)

    # load kakuho data from wgrib2 file
    grib_file = dt_now.strftime('%Y%m%d_%H%M00.000')
    source_folder = os.path.join(kakuho_folder, dt_now.strftime("%Y/%m/%d"))
    source_path = os.path.join(source_folder, grib_file)
    nc_file = source_path + ".nc"
    varlist = [":{} min".format(i) for i in range(5,65,5)]
    var = '|'.join(varlist)
    cmd = "wgrib2 {0} -s | egrep '({1})'|wgrib2 -i {0} -netcdf {2}".format(source_path, var, nc_file)
    fail = os.system(cmd)  # 0 indicate success, others indicate fail
    if fail:
        print("wgrib2 wrong at ", grib_file)
        continue

    root = Dataset(nc_file, "r")        
    # delete nc                     
    os.system(f"rm -r {nc_file}") 

    of_13_th = []  # threat score
    ka_13_th = [1]
    coverage_13 = []
    for i in range (13):
        dt_fcst = dt_now + timedelta(seconds = time_step * i)
        yday_fcst = dt_fcst.strftime('%Y_%m_%d')
        sequence = int(dt_fcst.hour*12 + (dt_fcst.minute)/5)   
        ground_truth =  daily_database[yday_fcst][sequence]/10.0
        of_13_th.append(metrics.CSI(ground_truth, nowcast[i], threshold = threshold))
        coverage = np.sum(ground_truth >= threshold)/(1e6 - 132396)*100
        coverage_13.append(coverage)
        if i >=1:
            rain_reduced = root['APCP_surface'][i-1,1500:2500,1000:2000]
            rain_reduced.fill_value = 0.0
            rain_filled = rain_reduced.filled().astype('float16')*6
            ka_13_th.append(metrics.CSI(ground_truth, rain_filled, threshold = threshold))
    plt.figure(dpi=75)
    plt.plot(lead_time, of_13_th,'o-',label = "Optical Flow (rainymotion-dense)")
    plt.plot(lead_time, ka_13_th,'s-',label = "Kakuho")
    plt.legend()
    plt.ylim([0,1])
    plt.grid()
    plt.ylabel("threat score")
    plt.xlabel("minutes after now")
    plt.title(f"now = {dt_now.strftime('%Y-%m-%d %H:%M')}, rain coverage = {coverage:.1f} %")
    plt.savefig(f"OFvsKakuho_{dt_now.strftime('%Y%m%d_%H%M')}.png",format = "png",bbox_inches='tight') 
    plt.close()

wgrib2 wrong at  20200122_050000.000
wgrib2 wrong at  20200122_060000.000
wgrib2 wrong at  20200122_070000.000
wgrib2 wrong at  20200122_080000.000
wgrib2 wrong at  20200122_090000.000
wgrib2 wrong at  20200122_100000.000
wgrib2 wrong at  20200122_110000.000
wgrib2 wrong at  20200122_120000.000
wgrib2 wrong at  20200122_130000.000
wgrib2 wrong at  20200122_140000.000
wgrib2 wrong at  20200122_150000.000
wgrib2 wrong at  20200122_160000.000
wgrib2 wrong at  20200122_170000.000
wgrib2 wrong at  20200122_180000.000
wgrib2 wrong at  20200122_190000.000
wgrib2 wrong at  20200122_200000.000
wgrib2 wrong at  20200122_210000.000
wgrib2 wrong at  20200122_220000.000


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jiang/data/jma_radar/jma_radar_uint8_2020_01_23.joblib'

In [74]:
import imageio
png_files = glob.glob("./OFvsKakuho*.png")
png_files.sort()
print(len(png_files)) # 288

images = []
for filename in png_files:
    images.append(imageio.imread(filename))
    os.system(f"rm -r {filename}")
output_file = f'OFvsKakuho-{yday_now}.gif'
imageio.mimsave(output_file, images,duration = 1)  # unit is in seconds

16


In [ ]:
# for download kakuho data or jma radar data
# source and goal folder should exist
from datetime import datetime,timedelta
import os
import urllib

# rain kakuho
base_URL  = "http://stock1.wni.co.jp/stock_hdd/411024220/0200600011024220"
base_goal = "/Users/jiang/data/rain_kakuho"

dt =     datetime(2020,1,22, 4,55)  # not included
dt_end = datetime(2020,1,23, 5,55)  # included
time_step = 5 * 60 # seconds
while dt < dt_end:
    dt += timedelta(seconds = time_step)
    file_string = dt.strftime('%Y%m%d_%H%M00.000')
    source_folder = os.path.join(base_URL, dt.strftime("%Y/%m/%d"))
    goal_folder = os.path.join(base_goal, dt.strftime("%Y/%m/%d"))
    # from pathlib import Path
    # Path("/my/directory").mkdir(parents=True, exist_ok=True)
    if not os.path.exists(goal_folder):
        os.makedirs(goal_folder)

    source_path = os.path.join(source_folder, file_string)
    goal_path = os.path.join(goal_folder, file_string)
    if os.path.isfile(goal_path):
        continue
    try: 
        urllib.request.urlretrieve(source_path)
    except: 
        print(f"not exist:{source_path}")
        continue


not exist:http://stock1.wni.co.jp/stock_hdd/411024220/0200600011024220/2020/01/22/20200122_065000.000
